In [2]:
import numpy as np
import plotly.graph_objects as go

from finmc.calc.iv import iv_surface_sim
from finmc.models.heston import HestonMC
from finmc.utils.assets import flat_discount, flat_fwds

In [3]:
asset_name = "SPX"
spot = 2900
rate = 0.02

dataset = {
    "MC": {
        "PATHS": 100_000,
        "TIMESTEP": 1 / 250,
        "SEED": 1,
    },
    "BASE": "USD",
    "ASSETS": {
        "USD": flat_discount(rate, 3.0),
        asset_name: flat_fwds(spot, rate, 0.0, 3.0),
    },
    "HESTON": {
        "ASSET": asset_name,
        "INITIAL_VAR": 0.02,
        "LONG_VAR": 0.04,
        "VOL_OF_VOL": 0.45,
        "MEANREV": 1.5,
        "CORRELATION": -0.85,
    },
}

In [4]:
# create the model and calculate the implied volatility surface
model = HestonMC(dataset)
strikes = np.linspace(2000, 3500, 16)
expirations = [1 / 12, 1 / 6, 1 / 4, 1 / 2, 1]
surface = iv_surface_sim(
    strikes,
    expirations,
    asset_name="SPX",
    model=model,
)

In [5]:
# Create the surface plot
fig = go.Figure(
    go.Surface(
        contours={"z": {"show": True, "start": 0.1, "end": 0.5, "size": 0.02}},
        z=surface,
        x=strikes,
        y=expirations,
    )
)

# Update layout for better visualization
fig.update_layout(
    scene=dict(
        xaxis_title="K",
        yaxis_title="Expirations",
        zaxis_title="Implied Vol",
        zaxis=dict(tickformat=".0%"),
    ),
    autosize=False,
    margin=dict(l=65, r=50, b=65, t=90),
)

In [22]:
fig = go.Figure()

# Add a line plot for each expiration
for i, expiration in enumerate(expirations):
    fig.add_trace(
        go.Scatter(
            x=strikes,
            y=surface[i, :],
            mode="lines",
            name=f"{expiration:0.2f}yr",
        )
    )

# Update layout for better visualization
fig.update_layout(
    xaxis_title="Strike Price (K)",
    yaxis=dict(title="Implied Volatility", tickformat=".0%"),
    autosize=False,
    legend_x=0.7,
    legend_y=0.98,
    margin=dict(l=65, r=50, b=65, t=10),
)